# 7. 합성곱 신경망
- CNN(convolutional neural network, CNN)은 이미지 인식과 음성 인식 등 당야한 곳에서 사용된다.
- 특히 이미지 인식 분야에서는 거의 다 CNN을 기초로 한다.

## 7.1 전체 구조
- 지금까지 본 신경망과 같이 레고 블록처럼 계층을 조합하여 만든다.
- 다만, 합성곱 계층과 풀링 계층이 등장한다.
- 지금까지 본 신경망은 인접하는 모든 뉴런과 결합되어 있었는데 이를 완전연결 이라고 하며, Affine 계층이라는 이름으로 구현했었다.
- CNN은 기존의 ( Affine > ReLU ) + (Affine > Softmax, sigmoid, 항등함수) 의 반복이 아닌, (Conv > ReLU > Pooling) + (Affine > ReLU) + (Affine > Softmax) 흐름으로 연결된다.

## 7.2 합성곱 계층

### 7.2.1 완전연결 계층의 문제점
- 완전연결 계층의 문제점은 '데이터의 형상이 무시'된다는 사실이다.
- 이미지는 3차원 형상이며, 3차원 속에서 의미를 갖는 본질적인 패턴이 숨어 있다.
- Conv 계층은 이미지도 3차원으로 받으며, 다음 계층에도 3차원으로 전달한다.
- CNN에서는 합성곱 계층의 입출력 데이터를 특징 맵(feature map) 이라고도 한다.

### 7.2.2 합성곱 연산
- 합성곱 연산은 이미지 처리에서 말하는 필터 연산에 해당한다.

### 7.2.3 패딩
- 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값으로 채우기도 한다.
- 이를 '패딩'이라고 하며, 합성곱 연산에서 자주 이용하는 기법이다.
- 패딩은 주로 출력 크기를 조정할 목적으로 사용한다.


### 7.2.4 스트라이드
- 필터를 적용하는 위치의 간격을 스트라이드라고 한다.
- 스트라이드를 2로 적용하면 필터가 한 번에 2칸을 기준으로 이동한다.

> 출력 크기 공식
- OH = {(입력 이미지 높이 + (2 * Padding) - 필터 가로) / 스트라이드 } + 1
- OW = {(입력 이미지 너비 + (2 * Padding) - 필터 너비)  스트라이드 } + 1
- 가급적 나눠떨어지게 구현해야한다.

### 7.2.5 3차원 이미지 합성곱 연산
- 필터의 채널 수를 입력데이터의 채널 수와 같도록 설정하고 해당 위치의 값들을 모두 더해서 하나의 출력값을 얻는다.

### 7.2.6 블록으로 생각하기
- 채널 수 C, 필터 높이 H, 너비 W인 데이터의 형상은 (C, H, W)로 쓰며 필터도 같은 순서로 (C, FH, FW)로 쓴다.
- 위 7.2.5단락의 3차원 이미지 합성곱 연산을 하게 되면 해당 3차원 이미지의 결과값으로 1차원으로 나오게 된다.
- 이를 방지하기 위해 필터를 다수 사용하게 된다.

<img src = './data/7_2_6.jpg' style="width:800px;height:300px;">

### 7.2.7 배치 처리
- 신경망에서는 입력 데이터를 한 덩어리로 묶어 배치로 처리했었다,
- 완전 신경망에서는 이러한 방식을 지원해 처리 효율을 높이고, 미니배치 방식의 학습도 지원하도록 했다.
- 합성곱 연산도 마찬가지로 배치 처치를 지원하는데 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차원 데이터로 저장한다.
- 데이터의 선두에 배치용 차원을 추가해 데이터는 4차원 형상을 가진 채 각 계층을 타고 흐른다.
- 신경망에 4차원의 데이터가 하나 흐를 때마다 데이터 N개에 대한 합성곱 연산이 이뤄진다. 즉, N회분의 처리를 한 번에 수행하는 것이다.

<img src = './data/7_2_7.jpg' style="width:800px;height:300px;">

## 7.3 풀링 계층
- 풀링은 세로, 가로 방향의 공간을 줄이는 연산이다.
- Max Pooling과 Average Pooling이 있다.
- 계산법은 알고 있기 때문에 생략
- 일반적으로 풀링과 스트라이드의 크기는 같게한다.

### 7.3.1 풀링 계층의 특징

- 학습해야할 매개 변수가 없다.
> 풀링 계층은 합성곱 계층과 달리 학습해야 할 매개변수가 없다.
- 채널 수가 변하지 않는다
> 채널마다 독립적으로 계산한다.
- 입력의 변화에 영향이 적다
> 입력데이터가 약간 변화해도 pooling에 의해 흡수된다.

## 7.4 합성곱 / 풀링 계층 구현하기

### 7.4.1 4차원 배열
- CNN 계층 사이를 흐르는 데이터는 4차원 이다.
- 예를 들어, 데이터의 형상이 (10,1,28,28) 이면, 이는 높이 28, 너비 28, 채널 1개인 데이터가 10개라는 말이다.

In [2]:
import numpy as np
x = np.random.rand(10,1,28,28)
x.shape

(10, 1, 28, 28)

In [5]:
print(x[0].shape)
print(x[1].shape)

(1, 28, 28)
(1, 28, 28)


### 7.4.2 im2col로 데이터 전개하기
- 합성곱 연산을 곧이곧대로 구현하려면 for 문을 겹겹이 써야한다.
- im2col이라는 함수를 사용해 구현
- im2col은 데이터를 필터링하기 좋게 전개하는 함수이다. 데이터 수 까지 포함한 4차원 데이터를 2차원으로 변환한다.

<img src = './data/7_4_2.jpg' style="width:800px;height:400px;">

### 7.4.3 합성곱 게층 구현하기

In [7]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [9]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride= 1,pad=0)
print(col1.shape)

(9, 75)


In [14]:
class Convolution:
    def __init__(self,W,b,stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH ) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW ) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        
        return out
    
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

### 7.4.4 풀링 계층 구현하기

In [15]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride = 1, pad = 0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx

## 7.7 대표적인 CNN

### 7.7.1 LeNet
- 1998년 제안된 손글씨 숫자를 인식하는 네트워크이다.
- 합성곱 계층과 풀링 계층(원소 줄이기만 하는 서브샘플링 계층)을 반복하고 마지막으로 affine 계층을 거치면서 결과를 출력한다.
- 시그모이드함수를 주로쓴다.

### 7.7.2 AlexNet
- LeNet과 큰 구조는 같다
- Activation Function 으로 ReLU를 쓴다
- LRN 이라는 국소적 정규화를 실시하는 계층을 이용한다.
- Drop Out을 사용한다.

Summary
- CNN 은 Fully Connected layer에 Convlutional layer + Pooling Layer를 추가한 구조이다.
- 계층이 깊어질수록 고급 정보다 추출된다.
- 대표적인 CNN에는 LeNet과 AlexNet가 있다.